In [1]:
import json
import pickle
import numpy as np
import pandas as pd
import os
import re
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn,Tensor
from typing import Union, Tuple, List, Iterable, Dict

In [2]:
import sys

In [3]:
sys.path.append("./deploy")
sys.path.append("./deploy/generation")

In [4]:
from modelInterface import modelInterface

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMaskedLM were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['cls.predictions.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
question_to_label = {"how to get above the office" : 1}
answer_to_label = {"Go right and then left" : 1}

In [6]:
data = {"question_to_label" : question_to_label , "answer_to_label" : answer_to_label}

In [6]:
covid_df = pd.read_csv("./data/covid19data/msf_covid19.csv", header = None)

In [7]:
covid_questions = {}
covid_answers = {}
label = 0
for q,a in zip(covid_df[1], covid_df[2]):
    q = q.split('\n')[0]
    a = a.split('\n')[0]
    if(a in covid_answers):
        l = covid_answers[a]
        covid_questions[q] = l
    else:
        covid_questions[q] = label
        covid_answers[a] = label
        label += 1


In [8]:
covid_data =  {"question_to_label" : covid_questions ,"answer_to_label" : covid_answers}

In [ ]:
modelInter = modelInterface("./FAQs/covid",covid_data)